In [ ]:
import os
import cv2
import xml.etree.ElementTree as ET

# Transform into 227x227

In [5]:
# Function to resize image and save it to a new folder
def resize_and_save(image_path, output_folder, size=(227, 227)):
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        print(f'Error loading image {image_path}')
        return

    # Resize the image
    resized_image = cv2.resize(image, size)

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Extract the filename from the image path
    filename = os.path.basename(image_path)

    # Create the full output path
    output_path = os.path.join(output_folder, filename)

    # Save the resized image
    cv2.imwrite(output_path, resized_image)
    print(f'Saved resized image to {output_path}')

# Example usage
input_folder = 'banana'
output_folder = 'banana_reshape'

# Get list of image files in the input folder
image_files = [f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))]

# Process each image file
for image_file in image_files:
    image_path = os.path.join(input_folder, image_file)
    resize_and_save(image_path, output_folder)

Saved resized image to banana_reshape\banana_1.jpg
Saved resized image to banana_reshape\banana_2.jpg


# Draw Bbox and create xml

In [9]:
# Initialize global variables for bounding box coordinates
bbox = {'xmin': 0, 'ymin': 0, 'xmax': 0, 'ymax': 0}
drawing = False

# Mouse callback function to draw bounding box
def draw_bbox(event, x, y, flags, param):
    global bbox, drawing

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        bbox['xmin'], bbox['ymin'] = x, y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            temp_image = image.copy()
            cv2.rectangle(temp_image, (bbox['xmin'], bbox['ymin']), (x, y), (255, 0, 0), 2)
            cv2.imshow('image', temp_image)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        bbox['xmax'], bbox['ymax'] = x, y
        cv2.rectangle(image, (bbox['xmin'], bbox['ymin']), (bbox['xmax'], bbox['ymax']), (255, 0, 0), 2)
        cv2.imshow('image', image)

# Function to create XML file
def create_xml(filename, folder, width, height, depth, bbox, object_name, save_path):
    annotation = ET.Element('annotation')

    folder_elem = ET.SubElement(annotation, 'folder')
    folder_elem.text = folder

    filename_elem = ET.SubElement(annotation, 'filename')
    filename_elem.text = filename

    source_elem = ET.SubElement(annotation, 'source')
    database_elem = ET.SubElement(source_elem, 'database')
    database_elem.text = 'MyDatabase'
    annotation_elem = ET.SubElement(source_elem, 'annotation')
    annotation_elem.text = 'COCO2017'
    image_elem = ET.SubElement(source_elem, 'image')
    image_elem.text = 'flickr'
    flickrid_elem = ET.SubElement(source_elem, 'flickrid')
    flickrid_elem.text = 'NULL'
    annotator_elem = ET.SubElement(source_elem, 'annotator')
    annotator_elem.text = '1'

    owner_elem = ET.SubElement(annotation, 'owner')
    flickrid_owner_elem = ET.SubElement(owner_elem, 'flickrid')
    flickrid_owner_elem.text = 'NULL'
    name_owner_elem = ET.SubElement(owner_elem, 'name')
    name_owner_elem.text = 'Label Studio'

    size_elem = ET.SubElement(annotation, 'size')
    width_elem = ET.SubElement(size_elem, 'width')
    width_elem.text = str(width)
    height_elem = ET.SubElement(size_elem, 'height')
    height_elem.text = str(height)
    depth_elem = ET.SubElement(size_elem, 'depth')
    depth_elem.text = str(depth)

    segmented_elem = ET.SubElement(annotation, 'segmented')
    segmented_elem.text = '0'

    object_elem = ET.SubElement(annotation, 'object')
    name_elem = ET.SubElement(object_elem, 'name')
    name_elem.text = object_name
    pose_elem = ET.SubElement(object_elem, 'pose')
    pose_elem.text = 'Unspecified'
    truncated_elem = ET.SubElement(object_elem, 'truncated')
    truncated_elem.text = '0'
    difficult_elem = ET.SubElement(object_elem, 'difficult')
    difficult_elem.text = '0'

    bndbox_elem = ET.SubElement(object_elem, 'bndbox')
    xmin_elem = ET.SubElement(bndbox_elem, 'xmin')
    xmin_elem.text = str(bbox['xmin'])
    ymin_elem = ET.SubElement(bndbox_elem, 'ymin')
    ymin_elem.text = str(bbox['ymin'])
    xmax_elem = ET.SubElement(bndbox_elem, 'xmax')
    xmax_elem.text = str(bbox['xmax'])
    ymax_elem = ET.SubElement(bndbox_elem, 'ymax')
    ymax_elem.text = str(bbox['ymax'])

    tree = ET.ElementTree(annotation)
    tree.write(save_path, encoding='utf-8', xml_declaration=True)

# Load the image
image_path = 'banana_reshape/banana_4.jpg'
image = cv2.imread(image_path)
cv2.namedWindow('image')
cv2.setMouseCallback('image', draw_bbox)

# Display the image and wait for the bounding box to be drawn
cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Extract filename from image path
filename = os.path.basename(image_path)

# Define other necessary details
folder = os.path.dirname(image_path)
height, width, depth = image.shape
object_name = 'banana'
save_path = save_path = os.path.splitext(filename)[0] + '.xml'

# Create the XML file
create_xml(filename, folder, width, height, depth, bbox, object_name, save_path)

In [2]:
filename

'4fbf5ecb-c02b4850-orange_middleright.jpg'